## NetLogo interaction through the pyNetLogo connector

This notebook provides a simple example of interaction between a NetLogo model and the Python environment, using the Wolf Sheep Predation model included in the NetLogo example library (Wilensky, 1999). A more detailed description of the different commands is provided in the paper.

We start by instantiating a link to NetLogo, loading the model, and executing the "setup" command in NetLogo.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import jpype
import pyNetLogo


netlogo = pyNetLogo.NetLogoLink(gui=True)

netlogo.load_model(r'Wolf Sheep Predation_v6.nlogo')
netlogo.command('setup')

We can use the write_NetLogo_attriblist function to pass properties to agents from a Pandas dataframe -- for instance, initial values for given attributes. This function improves performance by simultaneously setting multiple properties for multiple agents in a single function call.

As an example, we first load data from an Excel file into a dataframe. Each row corresponds to an agent, with columns for each attribute (including the "who" NetLogo identifier, which is required by the function). In this case, we set coordinates for the agents using the "xcor" and "ycor" attributes.

In [ ]:
agent_xy = pd.read_excel('xy_DataFrame.xlsx')
agent_xy[['who','xcor','ycor']].head(5)

In [ ]:
def write_NetLogo_attriblist(netlogo, agent_data, agent_name):
    '''
    Update a set of NetLogo agents of the same type with a list of attributes from a Pandas dataframe
    
    :param netlogo: NetLogoLink object
    :param agent_data: Dataframe with columns for each attribute to update, and a row for each agent. Requires a 'who'
                       column for the NetLogo agent ID
    :param agent_name: String for the name of the NetLogo agent type to update (singular, e.g. a-sheep
                        in the wolf-sheep predation example)
    '''

    #Get list of agent IDs to update, and convert to a Python string to pass to NetLogo
    whostr = ' '.join(map(str, agent_data['who']))

    #Name of NetLogo agent attributes to update
    attribs_to_update = [str(i) for i in agent_data.columns if i != 'who']

    #Convert the agent data to Python lists, then to strings to pass to NetLogo
    attribstr = []
    for attrib in attribs_to_update:
        #Check if we have string or numerical data in the dataframe and format the values accordingly
        if (agent_data[attrib].dtype == object):
            values = ' '.join(map(lambda x: '"{0}"'.format(x), list(agent_data[attrib])))
        else:
            values = ' '.join(map(str, list(agent_data[attrib]))) 
        #Then format the values string to look like a NetLogo list
        liststr=['[{0}]'.format(values)]
        attribstr.append(''.join(liststr))
        
    #Join the strings for all the attributes
    attribstr = ' '.join(attribstr)

    #Set up the "foreach" NetLogo command string
    askstr = []
    setstr = []
    for i, attrib_name in enumerate(attribs_to_update):
        askstr.extend(('?{0} '.format(i+2)))
        setstr.extend(('set {0} ?{1} '.format(attrib_name, i+2)))
    askstr = ''.join(askstr)
    setstr = ''.join(setstr)

    if netlogo.netlogo_version == '6':
        commandstr = ['(foreach [{0}] {1} [ [?1 {2}] -> ask {3} ?1 [{4}]])'.format(whostr, attribstr, askstr,
                                                                                   agent_name, setstr)]
    elif netlogo.netlogo_version == '5':
        commandstr = ['(foreach [{0}] {1} [ask {2} ?1 [{3}]])'].format(whostr, attribstr, agent_name, setstr)
        
    commandstr = ''.join(commandstr)
    
    netlogo.command(commandstr)

We can then pass the dataframe to NetLogo, specifying which attributes and which agent type we want to update:

In [ ]:
write_NetLogo_attriblist(netlogo, agent_xy[['who','xcor','ycor']], 'a-sheep')

We can check the data exchange by returning data from NetLogo to the Python workspace, using the report function. In the example below, the function returns arrays for the xcor and ycor coordinates of the "sheep" agents, sorted by their "who" number. These are then plotted on a conventional scatter plot.

The report function directly passes a string to the NetLogo instance, so that the command syntax may need to be adjusted depending on the NetLogo version. The netlogo_version property of the link object can be used to check the current version.

In [ ]:
if netlogo.netlogo_version == '6':
    x = netlogo.report('map [[?1] -> [xcor] of ?1] sort sheep')
    y = netlogo.report('map [[?1] -> [ycor] of ?1] sort sheep')
elif netlogo.netlogo_version == '5':
    x = netlogo.report('map [[xcor] of ?1] sort sheep')
    y = netlogo.report('map [[ycor] of ?1] sort sheep')

In [ ]:
sns.set_style('white')

fig, ax = plt.subplots(1)

ax.scatter(x, y, s=10)
ax.set_xlabel('xcor')
ax.set_ylabel('ycor')
ax.set_aspect('equal')

plt.show()

We can then run the model for 100 ticks and update the Python coordinate arrays for the sheep agents, and return an additional array for each agent's energy value. The latter is plotted on a histogram for each agent type.

In [ ]:
#We can use either of the following commands to run for 100 ticks:
netlogo.command('repeat 100 [go]')
#netlogo.repeat_command('go', 100)

if netlogo.netlogo_version == '6':
    x = netlogo.report('map [[?1] -> [xcor] of ?1] sort sheep')
    y = netlogo.report('map [[?1] -> [ycor] of ?1] sort sheep')
    energy_sheep = netlogo.report('map [[?1] -> [energy] of ?1] sort sheep')
elif netlogo.netlogo_version == '5':
    x = netlogo.report('map [[xcor] of ?1] sort sheep')
    y = netlogo.report('map [[ycor] of ?1] sort sheep')
    energy_sheep = netlogo.report('map [[energy] of ?1] sort sheep')
     
energy_wolves = netlogo.report('[energy] of wolves') #NetLogo returns these in random order

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 2)

sc = ax[0].scatter(x, y, s=50, c=energy_sheep, cmap=plt.cm.coolwarm)
ax[0].set_xlabel('xcor')
ax[0].set_ylabel('ycor')
ax[0].set_aspect('equal')
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.1)
cbar = plt.colorbar(sc, cax=cax, orientation='vertical')
cbar.set_label('Energy of sheep')

sns.distplot(energy_sheep, kde=False, bins=10, ax=ax[1], label='Sheep')
sns.distplot(energy_wolves, kde=False, bins=10, ax=ax[1], label='Wolves')
ax[1].set_xlabel('Energy')
ax[1].set_ylabel('Counts')
ax[1].legend()

fig.set_size_inches(12,5)
plt.show()

The repeat_report function returns a Pandas dataframe containing reported values over a given number of ticks, for one or multiple reporters. The dataframe is indexed by ticks, with labeled columns for each reporter. In this case, we track the number of wolf and sheep agents over 200 ticks; the outcomes are first plotted as a function of time. The number of wolf agents is then plotted as a function of the number of sheep agents, to approximate a phase-space plot. 

In [ ]:
counts = netlogo.repeat_report(['count wolves','count sheep'], 200)

fig, ax = plt.subplots(1, 2)

counts.plot(x=counts.index, ax=ax[0])
ax[0].set_xlabel('Ticks')
ax[0].set_ylabel('Counts')
ax[1].plot(counts['count wolves'], counts['count sheep'])
ax[1].set_xlabel('Wolves')
ax[1].set_ylabel('Sheep')

fig.set_size_inches(12,5)
plt.show()

The repeat_report function can also be used with reporters that return an array. In this case, we track the energy of the wolf and sheep agents over 5 ticks, and plot the distribution of the wolves' energy at the final tick recorded in the dataframe.

In [ ]:
energy_df = netlogo.repeat_report(['[energy] of wolves', '[energy] of sheep'], 5)

fig, ax = plt.subplots(1)

sns.distplot(energy_df['[energy] of wolves'].iloc[-1], kde=False, bins=20, ax=ax)
ax.set_xlabel('Energy')
ax.set_ylabel('Counts')

plt.show()

Finally, the patch_report function is used to return a dataframe which (for this example) contains the 'countdown' attribute of each NetLogo patch. This dataframe essentially replicates the NetLogo environment, with column labels corresponding to the xcor patch coordinates, and indices following the pycor coordinates.

In [ ]:
countdown_df = netlogo.patch_report('countdown')

fig, ax = plt.subplots(1)

patches = sns.heatmap(countdown_df, xticklabels=5, yticklabels=5, 
                      cbar_kws={'label':'countdown'}, ax=ax)
ax.set_xlabel('pxcor')
ax.set_ylabel('pycor')
ax.set_aspect('equal')

plt.show()

In [ ]:
netlogo.netlogo_version == '6'

The dataframes can be manipulated with any of the existing Pandas functions, for instance by exporting to an Excel file. The patch_set function provides the inverse functionality to patch_report, and updates the NetLogo environment from a dataframe.

In [ ]:
countdown_df.to_excel('countdown.xlsx')
netlogo.patch_set('countdown', countdown_df.max()-countdown_df)

In [ ]:
countdown_update_df = netlogo.patch_report('countdown')

fig, ax = plt.subplots(1)

patches = sns.heatmap(countdown_update_df, xticklabels=5, yticklabels=5, cbar_kws={'label':'countdown'}, ax=ax)
ax.set_xlabel('pxcor')
ax.set_ylabel('pycor')
ax.set_aspect('equal')

plt.show()

Finally, the kill_workspace() function shuts down the NetLogo instance.

In [ ]:
netlogo.kill_workspace()